In [4]:
# Import the PyTorch library
import torch

# Get the major and minor version of the current CUDA device (GPU)
major_version, minor_version = torch.cuda.get_device_capability()

# Apply the following if the GPU has Ampere or Hopper architecture (RTX 30xx, RTX 40xx, A100, H100, L40, etc.)
if major_version >= 8:
    # Install the Unsloth library for Ampere and Hopper architecture from GitHub
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git" -q

# Apply the following for older GPUs (V100, Tesla T4, RTX 20xx, etc.)
else:
    # Install the Unsloth library for older GPUs from GitHub
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" -q

# Placeholder statement (does nothing)
pass

# Install the Hugging Face Transformers library from GitHub, which allows native 4-bit loading
!pip install "git+https://github.com/huggingface/transformers.git" -q

!pip install trl datasets -q



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from unsloth import FastLanguageModel
# Import FastLanguageModel from the Unsloth library



max_seq_length = 2048  # Can be set arbitrarily, automatically supports RoPE scaling!
# Set the maximum sequence length to 2048 (can be changed arbitrarily)

dtype = None  # Automatically detect if None. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# Set the data type (automatically detect if None, can also specify Float16 or Bfloat16)

load_in_4bit = True  # Reduce memory usage using 4-bit quantization. Can be set to False.
# Reduce memory usage using 4-bit quantization (can be set to False to disable)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",  # Use "unsloth/mistral-7b" for 16-bit loading
    # Load the model "unsloth/mistral-7b-bnb-4bit" from pretraining (use "unsloth/mistral-7b" for 16-bit loading)

    max_seq_length=max_seq_length,
    # Set the maximum sequence length

    dtype=dtype,
    # Set the data type

    load_in_4bit=load_in_4bit,
    # Apply the 4-bit loading setting

    # token="hf_...",  # Use the token when using a gate model (e.g., meta-llama/Llama-2-7b-hf)
    # Use the Hugging Face token (when using a gate model, etc.)
)

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.1
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.691 GB
O^O/ \_/ \    CUDA capability = 8.6. Xformers = 0.0.22.post7. FA = True.
\        /    Pytorch version: 2.1.0+cu118. CUDA Toolkit = 11.8
 "-____-"     bfloat16 = TRUE. Platform = Linux

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

## LoRA adapter is added, and only updates 1-10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    # Specify the existing model

    r=16,  # Choose any positive number! Recommended values include 8, 16, 32, 64, 128, etc.
    # Rank parameter for LoRA. The smaller this value, the fewer parameters will be modified.

    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    # Specify the modules to which LoRA will be applied

    lora_alpha=16,
    # Alpha parameter for LoRA. This value determines the strength of the applied LoRA.

    lora_dropout=0,  # Currently, only supports dropout = 0
    # Dropout rate for LoRA. Currently supports only 0.

    bias="none",  # Currently, only supports bias = "none"
    # Bias usage setting. Currently supports only the setting without bias.

    use_gradient_checkpointing=True,
    # Whether to use gradient checkpointing to improve memory efficiency

    random_state=3407,
    # Seed value for random number generation

    max_seq_length=max_seq_length,
    # Set the maximum sequence length
)

Unsloth 2024.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Currently, we are using the Alpaca dataset from yahma. This is a filtered version of the original Alpaca dataset with 52K entries. You can replace this code section with your own data preparation.

In [6]:
#@title Alpaca Dataset Preparation Code

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
# Define the prompt format for the Alpaca dataset

def formatting_prompts_func(examples):
    # Define a function to format each example in the dataset

    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    # Get instructions, inputs, and outputs

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Generate text by combining instructions, inputs, and outputs

        text = alpaca_prompt.format(instruction, input, output)
        # Format the text according to the prompt format

        texts.append(text)
    return { "text" : texts, }
    # Return a list of formatted texts

pass
# Placeholder (does nothing)

from datasets import load_dataset
# Import the load_dataset function from the datasets library

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
# Load the training data of the cleaned version of the Alpaca dataset from yahma

dataset = dataset.map(formatting_prompts_func, batched=True,)
# Apply the formatting_prompts_func function to the dataset with batch processing


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

## Training the Model
Now, let's use the Hugging Face TRL's SFTTrainer! Details can be found here: TRL SFT Documentation. We will perform it in 60 steps to speed up, but you can set num_train_epochs=1 to run a full epoch, and turn off max_steps=None. Additionally, TRL's DPOTrainer is also supported!

In [7]:
from trl import SFTTrainer
# Import SFTTrainer from the TRL library

from transformers import TrainingArguments
# Import TrainingArguments from the Transformers library

trainer = SFTTrainer(
    # Initialize the SFTTrainer

    model=model,
    # Specify the model to be used

    train_dataset=dataset,
    # Specify the training dataset

    dataset_text_field="text",
    # Specify the text field in the dataset

    max_seq_length=max_seq_length,
    # Specify the maximum sequence length

    args=TrainingArguments(
        # Specify training arguments

        per_device_train_batch_size=2,
        # Specify the training batch size per device

        gradient_accumulation_steps=4,
        # Specify the number of steps for gradient accumulation

        warmup_steps=5,
        # Specify the number of warm-up steps

        max_steps=20,
        # Specify the maximum number of steps

        learning_rate=2e-4,
        # Specify the learning rate

        fp16=not torch.cuda.is_bf16_supported(),
        # Set whether to use 16-bit floating-point precision (fp16)

        bf16=torch.cuda.is_bf16_supported(),
        # Set whether to use Bfloat16

        logging_steps=1,
        # Specify the logging steps

        optim="adamw_8bit",
        # Specify the optimizer (here using 8-bit AdamW)

        weight_decay=0.01,
        # Specify the weight decay value

        lr_scheduler_type="linear",
        # Specify the type of learning rate scheduler (linear)

        seed=3407,
        # Specify the random seed

        output_dir="outputs",
        # Specify the output directory

    ),
)


tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
trainer.push_to_hub("Andyrasika/unsloth_alpaca")

CommitInfo(commit_url='https://huggingface.co/Andyrasika/outputs/commit/959bc3ea131908d07df8706d86719c9cf0f277f2', commit_message='Andyrasika/unsloth_alpaca', commit_description='', oid='959bc3ea131908d07df8706d86719c9cf0f277f2', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path = "outputs" ,
    repo_id = "Andyrasika/unsloth_alpaca" ,
    repo_type = "model"
)

CommitInfo(commit_url='https://huggingface.co/Andyrasika/unsloth_alpaca/commit/3ba92795076e78394ebf3a0f6288c34e042a7f6b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='3ba92795076e78394ebf3a0f6288c34e042a7f6b', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
#@title Display Current Memory Statistics

gpu_stats = torch.cuda.get_device_properties(0)
# Get properties of the GPU device at index 0

start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# Get the maximum reserved GPU memory in GB and round to 3 decimal places

max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# Get the total GPU memory in GB and round to 3 decimal places

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# Display the GPU name and maximum memory

print(f"{start_gpu_memory} GB of memory reserved.")
# Display the reserved memory amount


GPU = NVIDIA GeForce RTX 3090. Max memory = 23.691 GB.
4.625 GB of memory reserved.


In [9]:
#@title Execute the train method
trainer_stats = trainer.train()
# Execute the train method of the SFTTrainer, initiate training, and store training statistics in trainer_stats
# Note regarding Unsloth: `use_cache=True` is incompatible with gradient checkpointing, so it is set to `use_cache=False`.


Unsloth: `use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`


Step,Training Loss
1,1.403100
2,1.701900
3,1.190200
4,1.219200
5,1.015000
6,1.029800
7,1.047900
8,0.904000
9,0.945200
10,0.932300


In [10]:
#@title Display Final Memory and Time Statistics
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")


56.0791 seconds used for training.
0.93 minutes used for training.
Peak reserved memory = 5.836 GB.
Peak reserved memory for training = 1.211 GB.
Peak reserved memory % of max memory = 24.634 %.
Peak reserved memory for training % of max memory = 5.112 %.


In [11]:
#@title Conversion Code to GGUF

def colab_quantize_to_gguf(save_directory, quantization_method="q4_k_m"):
    # Define a function for conversion to GGUF

    from transformers.models.llama.modeling_llama import logger
    import os
    # Import necessary libraries

    logger.warning_once(
        "Unsloth: `colab_quantize_to_gguf` is still in development mode.\n"\
        "If anything errors or breaks, please file a ticket on Github.\n"\
        "Also, if you used this successfully, please tell us on Discord!"
    )
    # Warn that it's still in development mode and encourage reporting any issues

    # From https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html
    ALLOWED_QUANTS = \
    {
        # Define currently allowed quantization methods
        # Including descriptions for each quantization method
        "q2_k"   : "Uses Q4_K for the attention.vw and feed_forward.w2 tensors, Q2_K for the other tensors.",
        "q3_k_l" : "Uses Q5_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K",
        "q3_k_m" : "Uses Q4_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K",
        "q3_k_s" : "Uses Q3_K for all tensors",
        "q4_0"   : "Original quant method, 4-bit.",
        "q4_1"   : "Higher accuracy than q4_0 but not as high as q5_0. However has quicker inference than q5 models.",
        "q4_k_m" : "Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K",
        "q4_k_s" : "Uses Q4_K for all tensors",
        "q5_0"   : "Higher accuracy, higher resource usage and slower inference.",
        "q5_1"   : "Even higher accuracy, resource usage and slower inference.",
        "q5_k_m" : "Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K",
        "q5_k_s" : "Uses Q5_K for all tensors",
        "q6_k"   : "Uses Q8_K for all tensors",
        "q8_0"   : "Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.",
    }

    if quantization_method not in ALLOWED_QUANTS.keys():
        # If the specified quantization method is not allowed, raise an error
        error = f"Unsloth: Quant method = [{quantization_method}] not supported. Choose from below:\n"
        for key, value in ALLOWED_QUANTS.items():
            error += f"[{key}] => {value}\n"
        raise RuntimeError(error)

    # Display information about the conversion
    print_info = \
        f"==((====))==  Unsloth: Conversion from QLoRA to GGUF information\n"\
        f"   \\\   /|    [0] Installing llama.cpp will take 3 minutes.\n"\
        f"O^O/ \_/ \\    [1] Converting HF to GUUF 16bits will take 3 minutes.\n"\
        f"\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.\n"\
        f' "-____-"     In total, you will have to wait around 26 minutes.\n'
    print(print_info)
    # Display information about the conversion process

    if not os.path.exists("llama.cpp"):
        # If llama.cpp does not exist, install it
        print("Unsloth: [0] Installing llama.cpp. This will take 3 minutes...")
        !git clone https://github.com/ggerganov/llama.cpp
        !cd llama.cpp && make clean && LLAMA_CUBLAS=1 make -j
        !pip install gguf protobuf
        pass

    print("Unsloth: Starting conversion from HF to GGUF 16bit...")
    # Display that conversion from HF to GGUF 16bit is starting
    # print("Unsloth: [1] Converting HF into GGUF 16bit. This will take 3 minutes...")
    !python llama.cpp/convert.py {save_directory} \
        --outfile {save_directory}-unsloth.gguf \
        --outtype f16

    print("Unsloth: Starting conversion from GGUF 16bit to q4_k_m...")
    # Display that conversion from GGUF 16bit to the specified quantization method is starting
    # print("Unsloth: [2] Converting GGUF 16bit into q4_k_m. This will take 20 minutes...")
    final_location = f"./{save_directory}-{quantization_method}-unsloth.gguf"
    !./llama.cpp/quantize ./{save_directory}-unsloth.gguf \
        {final_location} {quantization_method}

    print(f"Unsloth: Output location: {final_location}")
    # Display the output location of the converted file
pass


In [12]:
from unsloth import unsloth_save_model
# Import the unsloth_save_model function from the Unsloth library

# unsloth_save_model has the same args as model.save_pretrained
# unsloth_save_model has the same arguments as model.save_pretrained
unsloth_save_model(model, tokenizer, "output_model", push_to_hub=False, token=None)
# Save the model and tokenizer as "output_model". Do not push to the Hugging Face Hub

colab_quantize_to_gguf("output_model", quantization_method="q4_k_m")
# Convert "output_model" to GGUF format. Use the quantization method "q4_k_m"


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model...

Unsloth: `colab_quantize_to_gguf` is still in development mode.
If anything errors or breaks, please file a ticket on Github.
Also, if you used this successfully, please tell us on Discord!


 Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'llama.cpp'...
remote: Enumerating objects: 16318, done.
remote: Counting objects: 100% (4281/4281), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 16318 (delta 4151), reused 4151 (delta 4072), pack-reused 12037
Receiving objects: 100% (16318/16318), 18.93 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (11343/11343), done.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=

...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
Unsloth: Starting conversion from HF to GGUF 16bit...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/workspace/llama.cpp/gguf-py
Traceback (most recent call last):
  File "/workspace/llama.cpp/convert.py", line 1662, in <module>
    main(sys.argv[1:])  # Exclude the first element (script name) from sys.argv
  File "/workspace/llama.cpp/convert.py", line 1581, in main
    model_plus = load_some_model(args.model)
  File "/workspace/llama.cpp/convert.py", line 1348, in load_some_model
    raise Exception(f"Can't find model in directory {path}")
Exception: Can't find model in directory output_model
Unsloth: Starting conversion from GGUF 16bit to q4_k_m...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
main: build = 1917 (57e2a7a)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './output_model-unsloth.gguf' to './output_model-q4_k_m-unsloth.gguf' as Q4_K_M
llama_model_quantize: failed to quantize: failed to open ./output_model-unsloth.gguf: No such file or directory
main: failed to quantize model from './output_model-unsloth.gguf'
Unsloth: Output location: ./output_model-q4_k_m-unsloth.gguf
